## Quick check any physics parameters for a single file

In [3]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, pathlib, shlex, subprocess
import xbout
import scipy
import xhermes
from xhermes import *


sys.path.append(r"/users/jpm590/2dspace/post-processing/sdtools/")


from hermes3.utils import *


from hermes3.fluxes import *
from hermes3.case_db import *
from hermes3.load import *
from hermes3.named_selections import *
from hermes3.plotting import *
from hermes3.grid_fields import *
from hermes3.accessors import *
from hermes3.selectors import *

%load_ext autoreload
%autoreload 2
print("Done")


Done


In [4]:
db = CaseDB(
    case_dir = r"/users/jpm590/scratch/",
    grid_dir = r"/users/jpm590/2dspace/hermes-3/build-mc-master"
)

toload = [
    dict(name="MAST-U", id="251105-pump-0.95", unnormalise_geom = True, use_xhermes = True, squash = True)
]
cs = {}
for case in toload:
    cs[case["name"]] = db.load_case_2D(case["id"], use_squash = case["squash"], verbose = True)
    cs[case["name"]].extract_2d_tokamak_geometry()

- Reading case 251105-pump-0.95
-----------------------
- Looking for squash file
- Squash file found. squash date 11/10/2025, 13:18:14, dmp file date 11/08/2025, 08:04:01

Unnormalising with xHermes


### Calculate the total power source
By `Pd+_src` + `Pe_src` times dv

Energy = 3/2 * pressure * volume 

`P_src` are in per second. So it's Power = 3/2 * P_src * dv

In [17]:
ds = cs["MAST-U"].ds.isel(t=-1)
power = 3/2 * (ds["Pd+_src"].values + ds["Pe_src"].values + ds["Pd_src"].values) * ds["dv"].values
power = np.sum(power)
print(f"Total power source = {power}")

print(ds["Pd_src"].values)
arr = ds["Pd_src"].values

# indices of non-zero elements
idx = np.nonzero(arr)
# or equivalently
idx = np.where(arr != 0)

print(f"At {idx}, the value is {ds["Pd_src"].values[idx]}")
# print(ds["dv"])

Total power source = 610442.265519238
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
At (array([37, 37]), array([21, 22])), the value is [1500779.77215842 1500779.77215842]


In [6]:
ds = cs["MAST-U"].ds.isel(t=-1)

nparticles = (ds["Sd_src"].values) * ds["dv"].values
print(nparticles)
nparticles = np.sum(nparticles)
print(f"Total number of particles = {nparticles}")

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Total number of particles = 5e+21
